In [1]:
! pip install datasets sacrebleu torch transformers 
! pip install sentencepiece transformers[sentencepiece]
! pip install accelerate -U
!pip install datasets==v2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess


## Installing Required Libraries

In [2]:

import warnings
import numpy as np
import pandas as pd

import torch
import transformers

from datasets import Dataset
#from datasets import load_metric

from tqdm import tqdm
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

warnings.filterwarnings("ignore")

## Helper Function

In [3]:
def postprocess_text(preds: list, labels: list) -> tuple:
    """Performs post processing on the prediction text and labels"""

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels



In [16]:
def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    """Takes the input data lists and converts into translation list of dicts"""

    data_dict = dict()
    data_dict[TRANSLATION] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict[FRENCH] = sr_text
        temp_dict[ENGLISH] = tr_text

        data_dict[TRANSLATION].append(temp_dict)

    return data_dict

In [5]:
def generate_model_ready_dataset(dataset: list, source: str, target: str,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer):
    """Makes the data training ready for the model"""

    preped_data = []

    for row in dataset:
        inputs = PREFIX + row[source]
        targets = row[target]

        model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)

        model_inputs[TRANSLATION] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)
            model_inputs[LABELS] = labels[INPUT_IDS]

        preped_data.append(model_inputs)

    return preped_data


In [6]:
def compute_metrics(eval_preds: tuple) -> dict:
    """computes bleu score and other performance metrics """

    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {BLEU: result[SCORE]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result[GEN_LEN] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [7]:

BATCH_SIZE = 16
BLEU = "bleu"
ENGLISH = "en"
ENGLISH_TEXT = "english_text"
EPOCH = "epoch"
INPUT_IDS = "input_ids"
FILENAME = "TranslationDataset.csv"
GEN_LEN = "gen_len"
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
MODEL_CHECKPOINT = "unicamp-dl/translation-pt-en-t5"
MODEL_NAME = MODEL_CHECKPOINT.split("/")[-1]
LABELS = "labels"
PREFIX = ""
FRENCH = "fr"
PORTUGUESE_TEXT = "portuguese_text"
SCORE = "score"
SOURCE_LANG = "pt"
TARGET_LANG = "en"
TRANSLATION = "translation"
UNNAMED_COL = "Unnamed: 0"

## Loading and Split The Dataset

In [8]:
input_texts = []
target_texts = []
with open("/kaggle/input/english-french-bilingual-sentence-pairs-232k/fra.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(12000  , len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    input_texts.append(input_text)
    target_texts.append(target_text)

In [9]:
input_texts[0],target_texts[1]

('Go.', 'Marche.')

In [10]:
# Split the data into train, test, and validation sets
x_train, x_test, y_train, y_test = train_test_split(input_texts, target_texts, test_size=0.10, shuffle=True, random_state=100)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, shuffle=True, random_state=100)

## Prepare the Model-Ready Dataset

In [17]:
# Prepare training, validation, and test data
training_data = prep_data_for_model_fine_tuning(x_train, y_train)
validation_data = prep_data_for_model_fine_tuning(x_val, y_val)
test_data = prep_data_for_model_fine_tuning(x_test, y_test)

# Generate model-ready inputs for training, validation, and test
train_data = generate_model_ready_dataset(dataset=training_data[TRANSLATION],
                                          tokenizer=tokenizer,
                                          source=FRENCH,
                                          target=ENGLISH,
                                          model_checkpoint=MODEL_CHECKPOINT)
validation_data = generate_model_ready_dataset(dataset=validation_data[TRANSLATION],
                                               tokenizer=tokenizer,
                                               source=FRENCH,
                                               target=ENGLISH,
                                               model_checkpoint=MODEL_CHECKPOINT)
test_data = generate_model_ready_dataset(dataset=test_data[TRANSLATION],
                                         tokenizer=tokenizer,
                                         source=FRENCH,
                                         target=ENGLISH,
                                         model_checkpoint=MODEL_CHECKPOINT)

In [18]:
# Convert to DataFrame
train_df = pd.DataFrame.from_records(train_data)
validation_df = pd.DataFrame.from_records(validation_data)
test_df = pd.DataFrame.from_records(test_data)

# Convert DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

## Load Tokenizer

In [19]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Load the pre-trained model and define training arguments
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

## Create Model Training Setup

In [20]:
model_args = Seq2SeqTrainingArguments(
    f"{MODEL_NAME}-finetuned-{SOURCE_LANG}-to-{TARGET_LANG}",
    evaluation_strategy=EPOCH,
    learning_rate=2e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True
)

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
# Initialize the Seq2SeqTrainer for fine-tuning
trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [22]:
# Commence the model training
trainer.train()

# Save the fine-tuned model
trainer.save_model("FineTunedTransformer")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.761200,1.862296
2,1.743100,1.454437
3,1.324200,1.271755
4,1.063200,1.195879
5,0.893900,1.178732
6,0.773400,1.169726
7,0.673000,1.165062
8,0.603500,1.171623
9,0.553600,1.184906
10,0.516900,1.191686


In [27]:
# Perform translations on the test dataset
test_results = trainer.predict(test_dataset)

# Obtain and display the test BLEU score
#print("Test Bleu Score: ", test_results.metrics["test_bleu"])

# Translate input sentences and generate predictions
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

predictions = []
test_input = test_dataset[TRANSLATION]

for input_text in tqdm(test_input):
    source_sentence = input_text[FRENCH]
    encoded_source = tokenizer(source_sentence,
                               return_tensors="pt",
                               padding=True,
                               truncation=True)
    encoded_source.to(device)  # Move input tensor to the same device as the model

    translated = model.generate(**encoded_source)

    predictions.append([tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])

# Move the model back to CPU if needed
model.to("cpu")

100%|██████████| 1200/1200 [03:08<00:00,  6.36it/s]


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [29]:
# Load the fine-tuned model and tokenizer
ft_model_tokenizer = T5Tokenizer.from_pretrained("FineTunedTransformer")
ft_model = T5ForConditionalGeneration.from_pretrained("FineTunedTransformer")

# Initialize an empty list for predictions
ft_prediction = []

# Translate Portuguese sentences to English using the fine-tuned model
for sentence in tqdm(x_test):
    encoded_text = ft_model_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    translated = ft_model.generate(**encoded_text)
    ft_prediction.append([tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])

100%|██████████| 1200/1200 [07:52<00:00,  2.54it/s]


In [31]:
ft_prediction[0],x_test[0]

('Tu es déterminé.', "You're driven.")